In [118]:
import csv

csvfile = open("decode.csv")
insts = csv.DictReader(csvfile)
insts = list(filter(lambda x: x["pat"] != "", insts))
for i in insts:
    i["name"] = i["name"].strip()
# reg_we
# op1_sel
# op2_sel
# op1_U
# op2_U
# alu_sel
# wb_sel
# invalid
# break

In [119]:
def fill_zero(inst):
    fields = ["invalid", "ebreak", "reg_we", "branch", "jal", "jalr", "mr", "mw"]
    for field in fields:
        inst[field] = int(inst[field] == "1")

In [120]:
def process_alu(inst):
    # alu_sel1/alu_sel2
    alu_sel1_dict = {"rs1": "00", "pc": "01", "zero": "10", "": "??"}
    alu_sel2_dict = {"rs2": "0", "imm": "1", "": "?"}
    inst["alu_sel1"] = alu_sel1_dict[inst["alu_sel1"]]
    inst["alu_sel2"] = alu_sel2_dict[inst["alu_sel2"]]

    # cmp_U
    if inst["cmp_U"] == "":
        inst["cmp_U"] = "?"

    # alu_func
    alu_func = [
        "add",
        "sub",
        "left",
        "right",
        "arith",
        "eq",
        "ne",
        "lt",
        "ge",
        "and",
        "or",
        "xor",
    ]
    alu_func_dict = {"": "????"}
    for i in range(len(alu_func)):
        alu_func_dict[alu_func[i]] = str(bin(i))[2:].zfill(4)
    inst["alu_func"] = alu_func_dict[inst["alu_func"]]

In [121]:
def process_mem(inst):
    # do nothing here
    # mem_len = inst(13,12)
    # load_U  = inst(14)
    pass

In [122]:
def process_wb(inst):
    wb_sel_dict = {"alu": "00", "mem": "01", "snpc": "10", "": "??"}
    inst["wb_sel"] = wb_sel_dict[inst["wb_sel"]]

In [123]:
def process_inst_type(inst):
    types_dict = {
        "I": "000",
        "U": "001",
        "S": "010",
        "B": "011",
        "J": "100",
        "R": "???",
        "": "???",
    }
    inst["inst_type"] = types_dict[inst["inst_type"]]
    # return inst

In [124]:
funcs = [
    # fill zeros
    fill_zero,
    # alu related signals: [alu_sel1, alu_sel2, alu_func, cmp_U]
    process_alu,
    # mem related signals: []
    process_mem,
    # write-back stage: [wb_sel]
    process_wb,
    # determine inst_type
    process_inst_type,
]
for i in insts:
    for f in funcs:
        f(i)
    for k, v in i.items():
        i[k] = str(v)
# insts = processed

In [125]:
for i in insts:
    print(i)

{'pat': '??????? ????? ????? ??? ????? 01101 11', 'name': 'lui', 'invalid': '0', 'ebreak': '0', 'reg_we': '1', 'alu_sel1': '10', 'alu_sel2': '1', 'cmp_U': '?', 'alu_func': '0000', 'branch': '0', 'jal': '0', 'jalr': '0', 'wb_sel': '00', 'inst_type': '001', 'mr': '0', 'mw': '0'}
{'pat': '??????? ????? ????? ??? ????? 00101 11', 'name': 'auipc', 'invalid': '0', 'ebreak': '0', 'reg_we': '1', 'alu_sel1': '01', 'alu_sel2': '1', 'cmp_U': '?', 'alu_func': '0000', 'branch': '0', 'jal': '0', 'jalr': '0', 'wb_sel': '00', 'inst_type': '001', 'mr': '0', 'mw': '0'}
{'pat': '??????? ????? ????? ??? ????? 11011 11', 'name': 'jal', 'invalid': '0', 'ebreak': '0', 'reg_we': '1', 'alu_sel1': '01', 'alu_sel2': '1', 'cmp_U': '?', 'alu_func': '0000', 'branch': '0', 'jal': '1', 'jalr': '0', 'wb_sel': '10', 'inst_type': '100', 'mr': '0', 'mw': '0'}
{'pat': '??????? ????? ????? ??? ????? 11001 11', 'name': 'jalr', 'invalid': '0', 'ebreak': '0', 'reg_we': '1', 'alu_sel1': '00', 'alu_sel2': '1', 'cmp_U': '?', 'al

In [126]:
class Signal:
    l: int = 0
    r: int = 0
    name: str = ""


offset = 0
signals = {}
for k, v in insts[0].items():
    if k == "pat" or k == "name":
        continue
    signal = Signal()
    signals[k] = signal
    signal.name = k
    signal.l = offset
    signal.r = offset + len(v) - 1
    offset += len(v)
    print(f"val {k} = UInt({len(v)}.W)")
control_len = offset

val invalid = UInt(1.W)
val ebreak = UInt(1.W)
val reg_we = UInt(1.W)
val alu_sel1 = UInt(2.W)
val alu_sel2 = UInt(1.W)
val cmp_U = UInt(1.W)
val alu_func = UInt(4.W)
val branch = UInt(1.W)
val jal = UInt(1.W)
val jalr = UInt(1.W)
val wb_sel = UInt(2.W)
val inst_type = UInt(3.W)
val mr = UInt(1.W)
val mw = UInt(1.W)


In [127]:
isa = {}
for i in insts:
    Value = str()
    for k, v in i.items():
        if k == "name" or k == "pat":
            continue
        Value += v
    print(f'BitPat("b{i["pat"]}") -> BitPat("b{Value}"),')
    i["control"] = Value
    isa[i["name"]] = {"pat": i["pat"], "ctl": Value}

BitPat("b??????? ????? ????? ??? ????? 01101 11") -> BitPat("b001101?00000000000100"),
BitPat("b??????? ????? ????? ??? ????? 00101 11") -> BitPat("b001011?00000000000100"),
BitPat("b??????? ????? ????? ??? ????? 11011 11") -> BitPat("b001011?00000101010000"),
BitPat("b??????? ????? ????? ??? ????? 11001 11") -> BitPat("b001001?00000011000000"),
BitPat("b??????? ????? ????? 000 ????? 11000 11") -> BitPat("b00000000101100??01100"),
BitPat("b??????? ????? ????? 001 ????? 11000 11") -> BitPat("b00000000110100??01100"),
BitPat("b??????? ????? ????? 100 ????? 11000 11") -> BitPat("b00000000111100??01100"),
BitPat("b??????? ????? ????? 101 ????? 11000 11") -> BitPat("b00000001000100??01100"),
BitPat("b??????? ????? ????? 110 ????? 11000 11") -> BitPat("b00000010111100??01100"),
BitPat("b??????? ????? ????? 111 ????? 11000 11") -> BitPat("b00000011000100??01100"),
BitPat("b??????? ????? ????? 000 ????? 00000 11") -> BitPat("b001001?00000000100010"),
BitPat("b??????? ????? ????? 001 ????? 0000

In [128]:
invalid = ["?" for i in range(control_len)]
invalid[signals["invalid"].l] = "1"
invalid = "".join(invalid)
isa["invalid"] = {"pat": "", "ctl": invalid}
print("b" + isa["invalid"]["ctl"])

b1????????????????????
